In [3]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
rawFile = '/Users/huangsir/Desktop/GEEX/模型/geex_credit_score/V01/train/data/raw/批量测试结果20210222/' # 原始数据
lgbFile = '/Users/huangsir/Desktop/GEEX/模型/ModelAPI/app/app/geex_credit_score/static/lgbm/' # lgb模型
lrFile = '/Users/huangsir/Desktop/GEEX/模型/ModelAPI/app/app/geex_credit_score/static/logit/' # lr模型

In [4]:
br_dt = pd.read_csv(''.join([rawFile,'百融/即科测试_结果.csv']),encoding='utf8',skiprows=0)
print(f'数据大小:{br_dt.shape}')
br_dt.head(2)

数据大小:(22997, 1425)


,cus_num,user_date,id,cell,name,swift_number,cus_username,code,flag_score,flag_inforelation,...,alu_y1_id_orgnum,alu_y1_cell_allnum,alu_y1_cell_orgnum,alu_fst_id_inteday,alu_lst_id_inteday,alu_fst_cell_inteday,alu_lst_cell_inteday,flag_fraudrelation_g,frg_list_level,frg_group_num
0,cus_num,user_date,id,cell,name,流水号,客户账号,返回码,flag_score,产品输出标识,...,1年以前，身份证号查询的申请机构数,1年以前，手机号查询的申请次数,1年以前，手机号查询的申请机构数,身份证号查询，最近申请距今天数,身份证号查询，最早申请距今天数,手机号查询，最近申请距今天数,手机号查询，最早申请距今天数,团伙欺诈排查通用版输出标识,查询人欺诈团伙等级,查询人欺诈团伙群组规模
1,2,2020/3/11,4370ba14897480c70b37a239dc14db52,a7da377985203f626a0fd050aa304203,a###############################,4002736_20210104152127_87775E0BA,demo13315,00,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,7,a


In [5]:
br_dict = {k:v for v,k in zip(br_dt.columns,br_dt.loc[0])}

In [22]:
with open(lgbFile+'features.txt','r') as f:
    lgb_f = f.read().split('\n')

lr_f =list(set(joblib.load(lrFile+'card.pkl')['variable']))

In [28]:
# 提取百融字段
br_f = {}
for f in lgb_f+lr_f:
    if 'br.' in f:
        if r'/' in f:
            v1,v2 = f.split('/')[0].strip(),f.split('/')[1].strip()
            key1,key2 = br_dict[v1.replace('br.','')],br_dict[v2.replace('br.','')]
            br_f[key1] = v1
            br_f[key2] = v2
        else:
            key = br_dict[f.replace('br.','')]
            br_f[key] = f
br_f

{'als_m3_id_pdl_allnum': 'br.按身份证号查询，近3个月申请线上小额现金贷的次数',
 'als_m12_cell_tot_mons': 'br.按手机号查询，近12个月有申请记录月份数',
 'als_m1_cell_pdl_allnum': 'br.按手机号查询，近1个月申请线上小额现金贷的次数',
 'als_m3_cell_pdl_orgnum': 'br.按手机号查询，近3个月申请线上小额现金贷的机构数',
 'als_m12_cell_caon_allnum': 'br.按手机号查询，近12个月申请线上现金分期的次数',
 'als_m6_id_nbank_nsloan_allnum': 'br.按身份证号查询，近6个月在非银机构-持牌网络小贷机构申请次数',
 'als_m12_cell_nbank_ca_allnum': 'br.按手机号查询，近12个月在非银机构-现金类分期机构申请次数',
 'als_m3_id_nbank_cf_allnum': 'br.按身份证号查询，近3个月在非银机构-消费类分期机构申请次数',
 'als_m12_id_nbank_sloan_allnum': 'br.按身份证号查询，近12个月在非银机构-持牌小贷机构申请次数',
 'als_m6_id_nbank_orgnum': 'br.按身份证号查询，近6个月在非银机构申请机构数',
 'als_m1_cell_nbank_nsloan_orgnum': 'br.按手机号查询，近1个月在非银机构-持牌网络小贷机构申请机构数',
 'als_m12_id_max_monnum': 'br.按身份证号查询，近12个月最大月申请次数',
 'als_m3_cell_nbank_cf_orgnum': 'br.按手机号查询，近3个月在非银机构-消费类分期申请机构数',
 'als_m3_cell_nbank_nsloan_orgnum': 'br.按手机号查询，近3个月在非银机构-持牌网络小贷机构申请机构数',
 'als_m12_cell_cooff_allnum': 'br.按手机号查询，近12个月申请线下消费分期的次数',
 'als_m3_id_nbank_cons_orgnum': 'br.按身份证号查询，近3个月在非银机构-持牌消费金融

In [29]:
# 同盾
# 贷前审核
td_sh = pd.read_excel(''.join([rawFile,'同盾/01贷前审核/01小额多期测试明细.xlsx']),skiprows=0)
td_sh.head(2)

,test_id,time_point,id_number,mobile,决策结果,Unnamed: 5,i_is_per_all_cease_courtcease_all_all,m1d_freq_ISEMULATOR_all_all_90day,i_is_per_all_tddefault_courtdefault_all_all,i_mom_cnt_30daypartner_Loan_Imbank_60day,...,i_is_per_all_exect_courtcease_all_all,m_is_suspectfakemobile,m1d_freq_SUSFAKINGLOCA_all_all_90day,i_is_per_self_all_taxowing_all_all,i_is_per_all_cease_courtdefault_all_all,success,i_cnt_partner_Loan_all_7day,i_cnt_partner_Loan_all_90day,i_ratio_cnt_partner_Loan_Offloan_90day,i_incr_set_recent90daypartner_Loan_Imbank_270day
0,测试序号,回溯日期,身份证号,手机号,决策建议,决策分数,身份证是否命中法院执行(案件状态为终本),手机号关联设备号二度关联到的90天内全行业下所有事件中设备为安卓模拟器的设备频次,身份证是否命中法院失信(案件状态为失信),身份证关联到的60天内非银行类金融行业下申贷事件中30日平台数的环比,...,身份证是否命中法院执行(案件状态为执行中),手机号是否命中疑似虚假号码,手机号关联设备号二度关联到的90天内全行业下所有事件中疑似模拟定位的设备频次,身份证是否命中欠税,身份证是否命中法院失信(案件状态为终本),调用结果,身份证关联到的7天内全行业下贷款事件中的不同平台个数,身份证关联到的90天内全行业下贷款事件中的不同平台个数,身份证关联到的90天内线下小额贷款行业下贷款事件中的不同平台个数占相应时间下所有不同平台个数的比例,身份证关联到的270天内非银行类金融行业下申贷事件中近90日的新增平台数
1,1,20200617,fe8d6ce9d8a3ccf198f0df6e6878eb95,27a1853ad9ce8ba1a9f3c21452d60f66,Review,63,-999,0,-999,25.0000,...,-999,-999,0,-999,-999,true,0,7,0.2857,5


In [30]:
td_dqsh_dict = {k:v for v,k in zip(td_sh.columns,td_sh.loc[0])}

In [35]:
# 同盾数据字典
td_f = {}
for f in lgb_f+lr_f:
    if 'td.dqsh.' in f:
        if r'/' in f:
            v1,v2 = f.split('/')[0].strip(),f.split('/')[1].strip()
            key1,key2 = td_dqsh_dict[v1.replace('td.dqsh.','')],td_dqsh_dict[v2.replace('td.dqsh.','')]
            td_f[key1] = v1
            td_f[key2] = v2
        else:
            key = td_dqsh_dict[f.replace('td.dqsh.','')]
            td_f[key] = f
            
    elif 'td.' in f:
        if 
        td_f[f.replace('td.zrf.','')] = f
td_f

{'智融分2.0通用分标准版': 'td.zrf.智融分2.0通用分标准版',
 '通用分标准版_': 'td.zrf.通用分标准版_',
 'i_incr_set_recent90daypartner_Loan_Imbank_270day': 'td.dqsh.身份证关联到的270天内非银行类金融行业下申贷事件中近90日的新增平台数',
 '智融分2.0非银分标准版': 'td.zrf.智融分2.0非银分标准版'}

In [46]:
for f in lgb_f+lr_f:
    if set(f.split('/'))&(set(br_f.values())|set(td_f.values())):
        if ('br.' not in f) and ('td.' not in f):
            print(f)